## Week 3 Assignment of Applied Data Science Capstone
#### Task 1: Transform the data on Wiki page into pandas dataframe

In [40]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

### Grep the wiki page content by using BeautifulSoup



In [41]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wiki_html = requests.get(url).text
soup = BeautifulSoup(wiki_html, 'html.parser')

In [42]:
# Convert content of PostalCode HTML table as list of data
data = []
for tr in soup.tbody.find_all('tr'):
    data.append([ td.get_text().strip() for td in tr.find_all('td')])
    
#Create the dataframe

df = pd.DataFrame(data, columns=['PostalCode','Borough','Neighborhood'])
df.head()


,PostalCode,Borough,Neighborhood
0,None,None,None
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


In [43]:
# Drop "None" rows

df = df.dropna()


In [44]:
# Drop any row which contains 'Not assigned' value
empty = 'Not assigned'
df = df[(df.PostalCode != empty ) & (df.Borough != empty) & (df.Neighborhood != empty)]
df.head()


,PostalCode,Borough,Neighborhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor


#### Group the dataframe by 'PostalCode' and 'Borough', convert the groupby value as string separated by commas and convert back to a new dataframe

In [45]:
def neighborhood_list(grouped):    
    return ', '.join(sorted(grouped['Neighborhood'].tolist()))
                    
grp = df.groupby(['PostalCode', 'Borough'])
df2 = grp.apply(neighborhood_list).reset_index(name='Neighborhood')

df2.head()


,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Highland Creek, Port Union, Rouge Hill"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [46]:
# Check some data
df2[df2.Borough == 'East York']

,PostalCode,Borough,Neighborhood
35,M4B,East York,"Parkview Hill, Woodbine Gardens"
36,M4C,East York,Woodbine Heights
38,M4G,East York,Leaside
39,M4H,East York,Thorncliffe Park
40,M4J,East York,East Toronto


In [47]:
# Save the dataframe to csv for future tasks
df2.to_csv('task1.csv', index=False)

In [49]:
#Print the shape of new dataframe
df2.shape


(102, 3)

## Task 2: Add the latitude and longitude coordinates to the dataframe

In [50]:
import pandas as pd


In [51]:
# Read the dataframe created in task 1

df = pd.read_csv('task1.csv')
df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Highland Creek, Port Union, Rouge Hill"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


#### Install geocoder library

In [52]:
!pip install geocoder
import geocoder

     |████████████████████████████████| 102kB 14.0MB/s ta 0:00:01


In [53]:

def get_latlng(postal_code):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    return lat_lng_coords
    
get_latlng('M4G')

[43.70949500000006, -79.36398897099997]

In [54]:
# Retrieve all Postal Code coordinates

postal_codes = df['PostalCode']    
coords = [ get_latlng(postal_code) for postal_code in postal_codes.tolist() ]

In [55]:
# Add Latitude and Longitude columns

df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
df['Latitude'] = df_coords['Latitude']
df['Longitude'] = df_coords['Longitude']

In [56]:
df.head()


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.811525,-79.195517
1,M1C,Scarborough,"Highland Creek, Port Union, Rouge Hill",43.785665,-79.158725
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.765815,-79.175193
3,M1G,Scarborough,Woburn,43.768369,-79.217590
4,M1H,Scarborough,Cedarbrae,43.769688,-79.239440


In [57]:
df[df.PostalCode == 'M5G']


,PostalCode,Borough,Neighborhood,Latitude,Longitude
57,M5G,Downtown Toronto,Central Bay Street,43.656091,-79.38493


In [58]:
# Save the dataframe for future

df.to_csv('task2.csv', index=False)
df


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.811525,-79.195517
1,M1C,Scarborough,"Highland Creek, Port Union, Rouge Hill",43.785665,-79.158725
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.765815,-79.175193
3,M1G,Scarborough,Woburn,43.768369,-79.217590
4,M1H,Scarborough,Cedarbrae,43.769688,-79.239440
5,M1J,Scarborough,Scarborough Village,43.743125,-79.231750
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.726276,-79.263625
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.713054,-79.285055
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.724235,-79.227925
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.696770,-79.259967
